# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [13]:
# import libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from IPython.core.display import HTML
from sqlalchemy import create_engine

%matplotlib inline

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [14]:
# create a Spark session
sc = SparkSession.builder.appName('capstone')\
    .master("local[*]").getOrCreate()

# enable sqlcontext
sqlContext = SQLContext(sc)

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [7]:
# file location
file_location = '../data/mini_sparkify_event_data.json'

# read in data
df = sc.read.json(file_location, multiLine=False)

In [16]:
df.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

In [8]:
df.count()

286500

In [9]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [19]:
df_sel = df.select("artist", "auth", "gender", "itemInSession", "length", "level", "location", "method", "page", "registration", "sessionId", "song", "status", "ts", "userAgent", "userId")

In [21]:
df_sel.describe("length", "itemInSession", "status").show()

+-------+------------------+------------------+------------------+
|summary|            length|     itemInSession|            status|
+-------+------------------+------------------+------------------+
|  count|            228108|            286500|            286500|
|   mean|249.11718197783375|114.41421291448516|210.05459685863875|
| stddev| 99.23517921058324|129.76726201141108| 31.50507848842204|
|    min|           0.78322|                 0|               200|
|    max|        3024.66567|              1321|               404|
+-------+------------------+------------------+------------------+



In [31]:
def get_distinct_vals(df):
    return df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)).toPandas().T.reset_index().rename(columns={'index':'column', 0:'n_distinct'})

def get_missing_vals_cnt(df):
    return df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T.reset_index().rename(columns={'index':'column', 0:'n_missing'})

In [32]:
missing_vals = get_missing_vals_cnt(df_sel)
distinct_vals = get_distinct_vals(df_sel)
missing_vals.merge(distinct_vals, on="column", how="inner")

,column,n_missing,n_distinct
0,artist,58392,17655
1,auth,0,4
2,gender,8346,2
3,itemInSession,0,1322
4,length,58392,14865
5,level,0,2
6,location,8346,114
7,method,0,2
8,page,0,22
9,registration,8346,225


In [59]:
df_sel.select(regexp_extract(col("userAgent"), "\((\w+\s)\)",0).alias("cleaned"), col("userAgent")).show()

+-------+--------------------+
|cleaned|           userAgent|
+-------+--------------------+
|       |Mozilla/5.0 (Wind...|
|       |"Mozilla/5.0 (Win...|
|       |Mozilla/5.0 (Wind...|
|       |"Mozilla/5.0 (Win...|
|       |Mozilla/5.0 (Wind...|
|       |"Mozilla/5.0 (Win...|
|       |"Mozilla/5.0 (Win...|
|       |Mozilla/5.0 (Wind...|
|       |Mozilla/5.0 (Wind...|
|       |Mozilla/5.0 (Wind...|
|       |"Mozilla/5.0 (Win...|
|       |"Mozilla/5.0 (Win...|
|       |Mozilla/5.0 (Wind...|
|       |"Mozilla/5.0 (Win...|
|       |"Mozilla/5.0 (Win...|
|       |Mozilla/5.0 (Wind...|
|       |"Mozilla/5.0 (Win...|
|       |"Mozilla/5.0 (Mac...|
|       |Mozilla/5.0 (Wind...|
|       |"Mozilla/5.0 (Win...|
+-------+--------------------+
only showing top 20 rows



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.